In [ ]:
# Google Colab에서 파일 업로드
from google.colab import files
import pandas as pd
import io

# 파일 업로드 실행
uploaded = files.upload()

# 업로드된 파일명 가져오기
file_name = list(uploaded.keys())[0]

# CSV 파일 로드
df = pd.read_csv(io.BytesIO(uploaded[file_name]))

# 데이터 확인
print("✅ 데이터 확인:")
print(df.head())


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# file_name 컬럼 제외하고 학습 데이터 생성
X = df.drop(columns=["file_name"]).values  # file_name 제거
y = df["type"].values  # 라벨 (0,1,2,3)

# 데이터를 학습용(train)과 테스트용(test)으로 분할 (80:20 비율)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 정규화 (StandardScaler 적용)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("✅ 데이터 전처리 완료")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 모델 생성
model = Sequential([
    Dense(16, activation='relu', input_shape=(X_train.shape[1],)),  # 입력층
    Dense(32, activation='relu'),  # 은닉층
    Dense(4, activation='softmax')  # 출력층 (4개의 클래스 → 다중 분류)
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # 다중 클래스 분류 손실 함수
              metrics=['accuracy'])

# 모델 요약 출력
model.summary()


In [ ]:
# 모델 학습
history = model.fit(X_train, y_train, epochs=50, batch_size=2, validation_data=(X_test, y_test))

# 학습 완료 후 모델 저장
model.save("malware_classification_model.h5")

print("✅ 모델 학습 및 저장 완료")


In [ ]:
import matplotlib.pyplot as plt

# 정확도 시각화
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.show()

# 손실값 시각화
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss')
plt.show()


In [ ]:
import numpy as np

# 모델 평가
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"\n✅ 테스트 정확도: {test_acc:.4f}")

# 샘플 데이터 예측
sample_input = X_test[:5]
predictions = model.predict(sample_input)
predicted_classes = np.argmax(predictions, axis=1)

print("\n실제 라벨:", y_test[:5])
print("예측 라벨:", predicted_classes)
